In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import scipy
from scipy.stats.qmc import Sobol
from scipy.stats import qmc
import matplotlib.pyplot as plt
import sys
sys.path.append('../../sample_utilities')
import samples
import json

In [6]:
target_volume = 1700
target_dilution = 0.1 # mol Si/L saxs sample
samples_write_fp = 'SampleTable_verifyReproducibility_5point7.csv'

### Calculate volumes from samples 

In [7]:
with open('constants_Round2_5point7.json', 'rt') as f:
    constants = json.load(f)

In [8]:
sample_1 = [0.018002, 0.027678, 0.009615]
sample_2 = [0.007526, 0.018891, 0.043859]

## Collate volume fractions into samples and get sample information

In [10]:
samples_gen = []
for sample_list in [sample_1, sample_2]:
    for i in range(3):
        sample = samples.SolidSilicaSample(target_volume=target_volume, reactant_fp='./constants_Round2_5point7.json', teos_vol_frac=sample_list[0], ammonia_vol_frac=sample_list[1], water_vol_frac=sample_list[2])
        sample.calculate_reactant_volumes()
        sample.calculate_silica_mass_concentration()
        sample.calculate_silica_mass_fraction()
        sample.calculate_dilution_volumefraction(target_dilution)
        samples_gen.append(sample)
        print('final etoh vol: ', sample.ethanol_volume)

441.80705
final etoh vol:  1164.1914499999998
441.80705
final etoh vol:  1164.1914499999998
441.80705
final etoh vol:  1164.1914499999998
561.50524
final etoh vol:  1019.02556
561.50524
final etoh vol:  1019.02556
561.50524
final etoh vol:  1019.02556


In [18]:
synth_table = samples.generate_synthesis_table(samples_gen)

In [19]:
synth_table

,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction
0,0be9d95b-b3d4-4246-99d9-cd5554c2ca13,174.43938,268.19982,93.16935,1164.19145,1.239942,4.845390,0.006080
1,6d4c6048-4b88-4a18-aed6-7ea936033cf0,174.43938,268.19982,93.16935,1164.19145,1.239942,4.845390,0.006080
2,01f260ef-7e01-4bd7-9bc6-712015100c9c,174.43938,268.19982,93.16935,1164.19145,1.239942,4.845390,0.006080
3,7b676039-761f-43c2-80e4-7a8bf22c1a35,72.92694,183.05379,424.99371,1019.02556,2.965909,2.025686,0.002527
4,3f4ea786-327e-4bd0-8ea4-1ad27b2d4187,72.92694,183.05379,424.99371,1019.02556,2.965909,2.025686,0.002527
5,f7c30e0d-fa5f-4d3a-a456-1319cc85efb0,72.92694,183.05379,424.99371,1019.02556,2.965909,2.025686,0.002527


In [20]:
rows = ['A', 'B', 'C', 'D', 'E', 'F']
cols = [1, 2, 3, 4, 5, 6, 7, 8]

wells = []
for row in rows:
    for col in cols:
        if row in ['B', 'E'] and col in [2, 7]:
            pass
        else:
            wells.append(row + str(col))

In [22]:
synth_table['well'] = wells[len(synth_table):2*len(synth_table)]

In [23]:
synth_table

,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,well
0,0be9d95b-b3d4-4246-99d9-cd5554c2ca13,174.43938,268.19982,93.16935,1164.19145,1.239942,4.845390,0.006080,A7
1,6d4c6048-4b88-4a18-aed6-7ea936033cf0,174.43938,268.19982,93.16935,1164.19145,1.239942,4.845390,0.006080,A8
2,01f260ef-7e01-4bd7-9bc6-712015100c9c,174.43938,268.19982,93.16935,1164.19145,1.239942,4.845390,0.006080,B1
3,7b676039-761f-43c2-80e4-7a8bf22c1a35,72.92694,183.05379,424.99371,1019.02556,2.965909,2.025686,0.002527,B3
4,3f4ea786-327e-4bd0-8ea4-1ad27b2d4187,72.92694,183.05379,424.99371,1019.02556,2.965909,2.025686,0.002527,B4
5,f7c30e0d-fa5f-4d3a-a456-1319cc85efb0,72.92694,183.05379,424.99371,1019.02556,2.965909,2.025686,0.002527,B5


In [24]:
synth_table.to_csv(samples_write_fp)